In [18]:
import requests
import json
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

ORION_URL = "http://localhost:1026"
MINTAKA_URL = "http://localhost:8080"

headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/ld+json',
        'Link': '<https://raw.githubusercontent.com/chzh63315/DigiEV/refs/heads/main/contexts/datamodels.context-ngsi.jsonld>; rel="http://www.w3.org/ns/json-ld#context"; type="application/ld+json"'
    }

In [19]:
# Check Orion Context Broker version
def test_connection():
    """Check if Orion Context Broker is accessible"""
    try:
        # Send GET request to version endpoint
        response = requests.get(f"{ORION_URL}/version")
        print(f"Status Code: {response.status_code}")
        
        if response.status_code == 200:
            version_info = response.json()
            print(f"Orion Version: {version_info.get('orion version', 'Unknown')}")
            print(f"Orion-LD Version: {version_info.get('orionld version', 'Unknown')}")
        else:
            print("Failed to connect to Orion")
            
    except Exception as e:
        print(f"Connection error: {e}")

# Run the test
test_connection()

Status Code: 200
Orion Version: 1.15.0-next
Orion-LD Version: 1.4.0


In [20]:
# def function for entity query test

def get_entity_types():
    """Get all available entity types"""
    try:
        response = requests.get(f"{ORION_URL}/ngsi-ld/v1/types", headers=headers)
        print(f"Status Code: {response.status_code}")
        if response.status_code == 200:
            types_data = response.json()
                         
            return types_data
        else:
            print(f"Failed: {response.text}")
            return []
    except Exception as e:
        print(f"Error: {e}")
        return []

def get_all_entities():
    """Get all entities from Orion Context Broker"""
    try:
        # Add local=true to avoid "too broad query" error
        params = {
            "limit": 1000,
            "local": "true"
        }
        
        response = requests.get(f"{ORION_URL}/ngsi-ld/v1/entities", 
                              headers=headers, 
                              params=params)
        print(f"Status Code: {response.status_code}")
        
        if response.status_code == 200:
            entities = response.json()
            print(f"Retrieved {len(entities)} entities")
            
            # Display basic info about each entity
            for i, entity in enumerate(entities):
                print(f"\nEntity {i+1}:")
                print(f"  ID: {entity.get('id')}")
                
                # Handle type field in NGSI-LD format
                entity_type = entity.get('type')
                if isinstance(entity_type, list):
                    entity_type = entity_type[0] if entity_type else 'Unknown'
                print(f"  Type: {entity_type}")
                
                # Count attributes (exclude @context, id, type)
                excluded_keys = ['@context', 'id', 'type']
                attr_count = len([k for k in entity.keys() if k not in excluded_keys])
                print(f"  Attributes: {attr_count}")
                
                # Show @context if present
                if '@context' in entity:
                    print(f"  Context: {entity.get('@context')}")
            
            return entities
        else:
            print("Failed to get entities")
            print(f"Response: {response.text}")
            return []
            
    except Exception as e:
        print(f"Error: {e}")
        return []
    
def query_entity_by_id(entity_id):
    """Query entity by exact ID"""
    try:
        response = requests.get(f"{ORION_URL}/ngsi-ld/v1/entities/{entity_id}", 
                              headers=headers)
        print(f"Status Code: {response.status_code}")
        
        if response.status_code == 200:
            entity = response.json()          
            return entity
        else:
            print(f"Entity not found: {response.text}")
            return None
            
    except Exception as e:
        print(f"Error: {e}")
        return None

def query_entities_by_type(entity_type):
    """Query entities by type"""
    try:
        params = {
            "type": entity_type,
            "limit": 10     # limit 10 outputs
        }
        
        response = requests.get(f"{ORION_URL}/ngsi-ld/v1/entities", 
                              headers=headers, 
                              params=params)
        print(f"Status Code: {response.status_code}")
        
        if response.status_code == 200:
            entities = response.json()
            print(f"Found {len(entities)} entities of type '{entity_type}'")
            
            for i, entity in enumerate(entities):
                print(f"\nEntity {i+1}:")
                print(f"  ID: {entity.get('id')}")
                print(f"  Type: {entity.get('type')}")
            
            return entities
        else:
            print(f"Query failed: {response.text}")
            return []
            
    except Exception as e:
        print(f"Error: {e}")
        return []

def query_entities_by_attribute(attribute_name, attribute_value):
    """Query entities by attribute value"""
    try:
        params = {
            "q": f'{attribute_name}=="{attribute_value}"',
            "limit": 10
        }
        
        response = requests.get(f"{ORION_URL}/ngsi-ld/v1/entities", 
                              headers=headers, 
                              params=params)
        print(f"Status Code: {response.status_code}")
        print(f"Query: {params['q']}")
        
        if response.status_code == 200:
            entities = response.json()
            print(f"Found {len(entities)} entities with {attribute_name}={attribute_value}")
            
            for i, entity in enumerate(entities):
                print(f"\nEntity {i+1}:")
                print(f"  ID: {entity.get('id')}")
                print(f"  Type: {entity.get('type')}")
                if attribute_name in entity:
                    attr_value = entity[attribute_name]
                    if isinstance(attr_value, dict) and 'value' in attr_value:
                        print(f"  {attribute_name}: {attr_value['value']}")
                    else:
                        print(f"  {attribute_name}: {attr_value}")
            
            return entities
        else:
            print(f"Query failed: {response.text}")
            return []
            
    except Exception as e:
        print(f"Error: {e}")
        return []



In [21]:
get_entity_types()

Status Code: 200


{'@context': 'https://raw.githubusercontent.com/chzh63315/DigiEV/refs/heads/main/contexts/datamodels.context-ngsi.jsonld',
 'id': 'urn:ngsi-ld:EntityTypeList:a8556fc0-5b16-11f0-8183-0242ac140005',
 'type': 'EntityTypeList',
 'typeList': ['ChargingPoint',
  'ChargingPointStatus',
  'ChargingSession',
  'ChargingSessionTest',
  'E-Vehicle']}

In [22]:
get_all_entities()

Status Code: 200
Retrieved 136 entities

Entity 1:
  ID: urn:ngsi-ld:ChargingPointStatus:CP_001_S_001
  Type: ChargingPointStatus
  Attributes: 4
  Context: https://raw.githubusercontent.com/chzh63315/DigiEV/refs/heads/main/contexts/datamodels.context-ngsi.jsonld

Entity 2:
  ID: urn:ngsi-ld:ChargingPoint:E4C-001-CP-001
  Type: ChargingPoint
  Attributes: 8
  Context: https://raw.githubusercontent.com/chzh63315/DigiEV/refs/heads/main/contexts/datamodels.context-ngsi.jsonld

Entity 3:
  ID: urn:ngsi-ld:E-Vehicle:EV-001
  Type: E-Vehicle
  Attributes: 8
  Context: https://raw.githubusercontent.com/chzh63315/DigiEV/refs/heads/main/contexts/datamodels.context-ngsi.jsonld

Entity 4:
  ID: urn:ngsi-ld:ChargingSession:session-512840826
  Type: ChargingSession
  Attributes: 14
  Context: https://raw.githubusercontent.com/chzh63315/DigiEV/refs/heads/main/contexts/datamodels.context-ngsi.jsonld

Entity 5:
  ID: urn:ngsi-ld:ChargingPointStatus:E4C-001-CP-001-S
  Type: ChargingPointStatus
  Attrib

[{'@context': 'https://raw.githubusercontent.com/chzh63315/DigiEV/refs/heads/main/contexts/datamodels.context-ngsi.jsonld',
  'id': 'urn:ngsi-ld:ChargingPointStatus:CP_001_S_001',
  'type': 'ChargingPointStatus',
  'dateCreated': {'type': 'Property',
   'value': {'@type': 'DateTime',
    '@value': '2025-06-30T09:20:23.035642+00:00'}},
  'operatingData': {'type': 'Property',
   'value': {'currentPowerOutput': 0,
    'currentVoltage': 230,
    'currentCurrent': 0,
    'powerFactor': 0.95,
    'efficiency': 92.5}},
  'refChargingPoint': {'type': 'Relationship',
   'object': 'urn:ngsi-ld:ChargingPoint:E4C-001-CP-001'},
  'unitStatus': {'type': 'Property', 'value': 'available'}},
 {'@context': 'https://raw.githubusercontent.com/chzh63315/DigiEV/refs/heads/main/contexts/datamodels.context-ngsi.jsonld',
  'id': 'urn:ngsi-ld:ChargingPoint:E4C-001-CP-001',
  'type': 'ChargingPoint',
  'chargingType': {'type': 'Property', 'value': 'AC-Level2'},
  'communicationProtocol': {'type': 'Property', 'va

In [23]:
query_entity_by_id("urn:ngsi-ld:ChargingSession:session-512840826")

Status Code: 200


{'@context': 'https://raw.githubusercontent.com/chzh63315/DigiEV/refs/heads/main/contexts/datamodels.context-ngsi.jsonld',
 'id': 'urn:ngsi-ld:ChargingSession:session-512840826',
 'type': 'ChargingSession',
 'refVehicleStatus': {'type': 'Relationship',
  'object': 'urn:ngsi-ld:E-VehicleStatus:vehicle-001-status'},
 'refChargingPoint': {'type': 'Relationship',
  'object': 'urn:ngsi-ld:ChargingPoint:E4C-001-CP-001'},
 'sessionId': {'type': 'Property', 'value': '512840826'},
 'sessionStatus': {'type': 'Property', 'value': 'charging'},
 'sessionType': {'type': 'Property', 'value': 'AC-Level2'},
 'sessionStartTime': {'type': 'Property',
  'value': {'@type': 'DateTime', '@value': '2025-06-30T11:21:04.759457Z'}},
 'chargingStartTime': {'type': 'Property',
  'value': {'@type': 'DateTime', '@value': '2025-06-30T11:21:34.759457Z'}},
 'sessionDuration': {'type': 'Property',
  'value': 25.5,
  'units': {'type': 'Property', 'value': 'MIN'}},
 'chargingDuration': {'type': 'Property',
  'value': 25,


In [24]:
query_entities_by_type("ChargingStation")


Status Code: 200
Found 0 entities of type 'ChargingStation'


[]

In [25]:
query_entities_by_attribute("sessionType", "AC-Level2")

Status Code: 200
Query: sessionType=="AC-Level2"
Found 10 entities with sessionType=AC-Level2

Entity 1:
  ID: urn:ngsi-ld:ChargingSession:session-512840826
  Type: ChargingSession
  sessionType: AC-Level2

Entity 2:
  ID: urn:ngsi-ld:ChargingSession:session-703497342
  Type: ChargingSession
  sessionType: AC-Level2

Entity 3:
  ID: urn:ngsi-ld:ChargingSession:session-412512340
  Type: ChargingSession
  sessionType: AC-Level2

Entity 4:
  ID: urn:ngsi-ld:ChargingSession:session-852888750
  Type: ChargingSession
  sessionType: AC-Level2

Entity 5:
  ID: urn:ngsi-ld:ChargingSession:session-718583992
  Type: ChargingSession
  sessionType: AC-Level2

Entity 6:
  ID: urn:ngsi-ld:ChargingSession:session-155986737
  Type: ChargingSession
  sessionType: AC-Level2

Entity 7:
  ID: urn:ngsi-ld:ChargingSession:session-487807670
  Type: ChargingSession
  sessionType: AC-Level2

Entity 8:
  ID: urn:ngsi-ld:ChargingSession:session-528124502
  Type: ChargingSession
  sessionType: AC-Level2

Entity 9:

[{'@context': 'https://raw.githubusercontent.com/chzh63315/DigiEV/refs/heads/main/contexts/datamodels.context-ngsi.jsonld',
  'id': 'urn:ngsi-ld:ChargingSession:session-512840826',
  'type': 'ChargingSession',
  'authenticationId': {'type': 'Property', 'value': '1'},
  'authenticationMethod': {'type': 'Property', 'value': 'RFID'},
  'chargingDuration': {'type': 'Property',
   'value': 25,
   'units': {'type': 'Property', 'value': 'MIN'}},
  'chargingStartTime': {'type': 'Property',
   'value': {'@type': 'DateTime', '@value': '2025-06-30T11:21:34.759457Z'}},
  'dateCreated': {'type': 'Property',
   'value': {'@type': 'DateTime', '@value': '2025-06-30T11:21:04.759457Z'}},
  'lastUpdated': {'type': 'Property',
   'value': {'@type': 'DateTime', '@value': '2025-06-30T11:21:04.759779Z'}},
  'refChargingPoint': {'type': 'Relationship',
   'object': 'urn:ngsi-ld:ChargingPoint:E4C-001-CP-001'},
  'refVehicleStatus': {'type': 'Relationship',
   'object': 'urn:ngsi-ld:E-VehicleStatus:vehicle-001-


Headers:
Content-Type: application/json
Accept: application/ld+json
Link: <https://raw.githubusercontent.com/chzh63315/DigiEV/refs/heads/main/contexts/datamodels.context-ngsi.jsonld>; rel="http://www.w3.org/ns/json-ld#context"; type="application/ld+json"


query session duration > 500

Get

http://localhost:1026/ngsi-ld/v1/entities?type=ChargingSession&q=sessionDuration>500&limit=20



http://localhost:8080/temporal/entities/?type=ChargingSession&timerel=after&timeAt=2024-12-31T23:59:59Z&timeproperty=observedAt&attrs=sessionStartTime&limit=20

http://localhost:8080/temporal/entities/?type=ChargingSession&timerel=after&timeAt=2024-12-31T23:59:59Z&timeproperty=observedAt&attrs=energyConsumed&limit=20

http://localhost:8080/temporal/entities/?type=ChargingSession&timerel=after&timeAt=2024-12-31T23:59:59Z&timeproperty=observedAt&attrs=energyConsumed,chargingDuration&limit=20

http://localhost:8080/temporal/entities/?type=ChargingSession&timerel=between&timeAt=2025-03-01T00:00:00Z&endTimeAt=2025-05-31T23:59:59Z&timeproperty=observedAt&attrs=sessionEndTime&limit=100



